In [1]:
import csv
from pyspark import SparkContext
from collections import Counter
from itertools import combinations

In [2]:
c = Counter()
c.update(['101_102'])
c.update(['101_104'])
c.update(['101_99'])
c

Counter({'101_102': 1, '101_104': 1, '101_99': 1})

In [59]:
def test_loop(boolean_list):
    for i in boolean_list:
        if i is False:
            return False
    return True

def test_all(boolean_list):
    return all(boolean_list)

l = [True,False,True,True,False,False]
r = [True,True]

max_len = 10000
boolean_list = [True if random.random()<0.7 else False for i in range(max_len)]

%timeit test_loop(boolean_list)
%timeit test_all(boolean_list)

125 ns ± 3.56 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
111 ns ± 1.73 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [2]:
input_file = "../data/small2.csv"
support = 9

In [3]:
raw_data = [] # user_id,business_id
with open(input_file,'r') as input:
    readCSV = csv.reader(input, delimiter=',')
    cnt = 0
    for row in readCSV:
        cnt += 1
        if cnt >1:
            raw_data.append(row)

baskets = dict()  # user_id: [business_id1, business_id2, business_id3, ...]
if case==1:
    for row in raw_data:
        if row[0] not in baskets:
            baskets[row[0]] = []
        baskets[row[0]].append(row[1])
elif case==2:
    for row in raw_data:
        if row[1] not in baskets:
            baskets[row[1]] = []
        baskets[row[1]].append(row[0])

In [98]:
for user in baskets:
    basket = baskets[user]
    baskets[user] = sorted(basket)
clean_baskets = [baskets[i] for i in baskets]

single_cnt = Counter()
for basket in clean_baskets:
    single_cnt.update(basket)
    
    
max_itemset_size = max([len(basket) for basket in clean_baskets])

candidate_single = sorted([i for i in single_cnt if single_cnt[i]>=support])
former_candidate = candidate_single
print(candidate_single)

candidate_itemsets = {str(i):[] for i in range(1,max_itemset_size+1)}
candidate_itemsets['1'].extend(former_candidate)

for itemset_size in range(2,max_itemset_size+1):
    tmp_itemsets = []
    for basket in clean_baskets:
        if len(basket) < itemset_size:
            continue
        for itemset in combinations(basket,itemset_size):
            itemset = sorted(itemset)
            if itemset_size == 2:
                immediate_subset = [i[0] for i in combinations(itemset,itemset_size-1)]
            else:
                immediate_subset = [sorted(i) for i in combinations(itemset,itemset_size-1)]
            immediate_subset_is_candidate = [i in former_candidate for i in immediate_subset]
            if all(immediate_subset_is_candidate):
                tmp_itemsets.append('_'.join(itemset))
    itemset_cnt = Counter(tmp_itemsets)
    former_candidate = [i.split('_') for i in itemset_cnt if itemset_cnt[i]>=support]
    candidate_itemsets[str(itemset_size)].extend(former_candidate)

['100', '101', '102', '97', '98', '99']


In [96]:
candidate_itemsets

{'1': ['100', '101', '102', '97', '98', '99'],
 '2': [['100', '101'],
  ['101', '102'],
  ['102', '98'],
  ['101', '99'],
  ['97', '99'],
  ['97', '98'],
  ['98', '99']],
 '3': [['97', '98', '99']],
 '4': [],
 '5': [],
 '6': [],
 '7': [],
 '8': [],
 '9': [],
 '10': []}

In [82]:
def market_basket(input_file,support,case,verbose=False):
    raw_data = [] # user_id,business_id
    with open(input_file,'r') as input:
        readCSV = csv.reader(input, delimiter=',')
        cnt = 0
        for row in readCSV:
            cnt += 1
            if cnt >1:
                raw_data.append(row)

    item_frequency = dict()
    if case == 1:
        for row in raw_data:
            if row[1] not in item_frequency:
                item_frequency[row[1]] = 1
            else:
                item_frequency[row[1]] += 1
    elif case == 2:
        for row in raw_data:
            if row[0] not in item_frequency:
                item_frequency[row[0]] = 1
            else:
                item_frequency[row[0]] += 1

    frequent_item = []
    for item in item_frequency:
        if item_frequency[item] >= support:
            frequent_item.append(item)


    if verbose is True:
        print("\nFrequent items: ",frequent_item)

    baskets = dict()  # user_id: [business_id1, business_id2, business_id3, ...]
    if case==1:
        for row in raw_data:
            if row[0] not in baskets:
                baskets[row[0]] = []
            baskets[row[0]].append(row[1])
    elif case==2:
        for row in raw_data:
            if row[1] not in baskets:
                baskets[row[1]] = []
            baskets[row[1]].append(row[0])

    for user in baskets:
        basket = baskets[user]
        baskets[user] = sorted(basket)


    pair_cnt = dict()
    frequent_pair = []
    for user in baskets:
        basket = baskets[user]
        basket_len = len(basket)
        for i in range(basket_len-1):
            for j in range(i+1, basket_len):
                pair = basket[i] + '_' + basket[j]
                if pair not in pair_cnt:
                    pair_cnt[pair] = 0
                pair_cnt[pair] += 1

    for pair in pair_cnt:
        if pair_cnt[pair] >= support:
            double_itemset = pair.split('_')
            frequent_pair.append(double_itemset)
    if verbose is True:
        print("\nFrequent pairs: ", frequent_pair)

    frequent_pair_string = []
    for pair in frequent_pair:
        pair_string = '_'.join(pair)
        frequent_pair_string.append(pair_string)

    triple_cnt = dict()
    for user in baskets:
        basket = baskets[user]
        basket_len = len(basket)
        for i in range(basket_len-2):
            for j in range(i+1, basket_len-1):
                for k in range(j+1, basket_len):
                    p1 = basket[i] + '_' + basket[j]
                    p2 = basket[j] + '_' + basket[k]
                    p3 = basket[i] + '_' + basket[k]
                    if p1 in frequent_pair_string and p2 in frequent_pair_string and p3 in frequent_pair_string:
                        triple =  basket[i] + '_' + basket[j] + '_' + basket[k]
                        if triple not in triple_cnt:
                            triple_cnt[triple] = 0
                        triple_cnt[triple] += 1
    
    frequent_triple = []
    for triple in triple_cnt:
        if triple_cnt[triple] >= support:
            frequent_triple.append(triple.split("_"))
    if verbose is True:
        print("\nFrequent triples: ", frequent_triple)


    candidate_after_triple = []
    for i in frequent_triple:
        candidate_after_triple.extend(i)    

    candidate_after_triple = sorted(list(set(candidate_after_triple)))
    

    four_cnt = dict()
    frequent_triple_string = []
    for i in frequent_triple:
        frequent_triple_string.append('_'.join(i))
    for user in baskets:
        basket = baskets[user]
        basket_len = len(basket)
        for i in range(basket_len-3):
            for j in range(i+1, basket_len-2):
                for k in range(j+1, basket_len-1):
                    for m in range(k+1, basket_len):
                        p1 = basket[i] + '_' + basket[j] + '_' + basket[k]
                        p2 = basket[j] + '_' + basket[k] + '_' + basket[m]
                        p3 = basket[i] + '_' + basket[j] + '_' + basket[m]
                        p4 = basket[j] + '_' + basket[k] + '_' + basket[m]
                        if p1 in frequent_triple_string and p2 in frequent_triple_string and p3 in frequent_triple_string:
                            if p4 in frequent_triple_string: 
                                four =  basket[i] + '_' + basket[j] + '_' + basket[k] + '_' + basket[m]
                                if four not in four_cnt:
                                    four_cnt[four] = 0
                                four_cnt[four] += 1
    
    
    frequent_four = []
    for four in four_cnt:
        if four_cnt[four] >= support:
            frequent_four.append(four.split("_"))
            
    
    five_cnt = []
    
    
    return {"singleton": frequent_item, "pair": frequent_pair, "triple": frequent_triple, "four":frequent_four}

In [63]:
%timeit  market_basket(input_file,support)

659 µs ± 11.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [93]:
market_basket(input_file,5,True)


Frequent items:  ['100', '98', '101', '102', '99', '97']

Frequent pairs:  [['100', '101'], ['101', '102'], ['102', '98'], ['101', '99'], ['97', '99'], ['97', '98'], ['98', '99']]

Frequent triples:  [['97', '98', '99']]


{'singleton': ['100', '98', '101', '102', '99', '97'],
 'pair': [['100', '101'],
  ['101', '102'],
  ['102', '98'],
  ['101', '99'],
  ['97', '99'],
  ['97', '98'],
  ['98', '99']],
 'triple': [['97', '98', '99']],
 'four': []}

In [65]:
%timeit  result = market_basket("../data/small2.csv",6)

216 ms ± 8.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
result = market_basket("../data/small2.csv",6)

In [69]:
market_basket("../data/small2.csv",10,True)


Frequent items:  ['107', '106', '103', '97', '105', '100', '101', '102', '99', '116', '111', '112', '115', '108', '104', '98', '110', '113', '114', '109']

Frequent pairs:  [['100', '103'], ['100', '105'], ['100', '106'], ['100', '107'], ['100', '97'], ['103', '105'], ['103', '106'], ['103', '107'], ['103', '97'], ['105', '106'], ['105', '107'], ['105', '97'], ['106', '107'], ['106', '97'], ['107', '97'], ['100', '101'], ['100', '102'], ['100', '99'], ['101', '102'], ['101', '103'], ['101', '106'], ['101', '107'], ['101', '97'], ['101', '99'], ['102', '103'], ['102', '106'], ['102', '107'], ['102', '111'], ['102', '112'], ['102', '97'], ['102', '99'], ['103', '111'], ['103', '112'], ['103', '99'], ['106', '111'], ['106', '99'], ['107', '111'], ['107', '112'], ['107', '99'], ['112', '99'], ['97', '99'], ['100', '104'], ['100', '108'], ['101', '104'], ['101', '108'], ['103', '104'], ['103', '108'], ['104', '106'], ['104', '107'], ['104', '108'], ['104', '97'], ['106', '108'], ['107', '1

{'singleton': ['107',
  '106',
  '103',
  '97',
  '105',
  '100',
  '101',
  '102',
  '99',
  '116',
  '111',
  '112',
  '115',
  '108',
  '104',
  '98',
  '110',
  '113',
  '114',
  '109'],
 'pair': [['100', '103'],
  ['100', '105'],
  ['100', '106'],
  ['100', '107'],
  ['100', '97'],
  ['103', '105'],
  ['103', '106'],
  ['103', '107'],
  ['103', '97'],
  ['105', '106'],
  ['105', '107'],
  ['105', '97'],
  ['106', '107'],
  ['106', '97'],
  ['107', '97'],
  ['100', '101'],
  ['100', '102'],
  ['100', '99'],
  ['101', '102'],
  ['101', '103'],
  ['101', '106'],
  ['101', '107'],
  ['101', '97'],
  ['101', '99'],
  ['102', '103'],
  ['102', '106'],
  ['102', '107'],
  ['102', '111'],
  ['102', '112'],
  ['102', '97'],
  ['102', '99'],
  ['103', '111'],
  ['103', '112'],
  ['103', '99'],
  ['106', '111'],
  ['106', '99'],
  ['107', '111'],
  ['107', '112'],
  ['107', '99'],
  ['112', '99'],
  ['97', '99'],
  ['100', '104'],
  ['100', '108'],
  ['101', '104'],
  ['101', '108'],
  ['103

In [3]:
input_file = "../data/small1.csv"
support = 4
case = 1


sc = SparkContext.getOrCreate()
raw_data = sc.textFile(input_file)
header = raw_data.first()
raw_data_without_header = raw_data.filter(lambda x: x != header)
clean_data = raw_data_without_header.map(lambda line: [i for i in line.strip().split(',')])

# whether there is faster method??
if case==1:
    baskets = clean_data.map(lambda x: (x[0], [x[1]])).reduceByKey(lambda a, b: a + b).map(
            lambda x: [x[0], sorted(list(set(list(x[1]))))])
elif case==2:
    baskets = clean_data.map(lambda x: (x[1], [x[0]])).reduceByKey(lambda a, b: a + b).map(
            lambda x: [x[0], sorted(list(set(list(x[1]))))])

# get rid of original key:
#   case 1 only remain [business_ids];
#   case 2 only remain [user_ids]
clean_baskets = baskets.map(lambda x: x[1]).collect()  # TODO
sc.stop()

In [9]:
# clean_baskets = list(baskets)
# local_support = support * len(clean_baskets) / float(total_baskets_cnt)
baskets_cnt_ordered = sorted(Counter([len(basket) for basket in clean_baskets]).items(),reverse=True)
print("Count of Basket with different length:",baskets_cnt_ordered)
# Reduce max_itemset_size, if the number of baskets with certain length is less than local_support,
# then the maximum size of frequent itemset must be smaller than the length of such baskets
acc_sum = 0
for cnt_item in baskets_cnt_ordered:
    max_itemset_size = cnt_item[0]
    acc_sum += cnt_item[1]
    if acc_sum >= support:
        break
    else:
        continue
print("Maximum candidate frequent itemset size:%d" % max_itemset_size)
candidate_itemsets = []

# frequent items
single_cnt = Counter()
for basket in clean_baskets:
    single_cnt.update(basket)

candidate_single = sorted([i for i in single_cnt if single_cnt[i] >= support])
former_candidate = candidate_single
candidate_itemsets.extend(former_candidate)

# frequent pairs
itemset_size = 2
itemset_cnt = Counter()
former_possible_items_set = former_candidate
for pair in combinations(former_possible_items_set,2):
    pair = sorted(pair)
    for basket in clean_baskets:
        contain_pair_flag = True
        if len(basket) < 2:
            continue
        for item in pair:
            if item not in basket:
                contain_pair_flag = False
                break
        if contain_pair_flag:
            itemset_cnt.update(['_'.join(pair)])
former_candidate = [i for i in itemset_cnt if itemset_cnt[i] >= support]
candidate_itemsets.extend(former_candidate)
if len(former_candidate)<2:
    print("True possible maximum frequent itemset size: %d" % itemset_size)

Count of Basket with different length: [(10, 1), (8, 1), (6, 2), (5, 1), (4, 3), (3, 6), (2, 4), (1, 1)]
Maximum candidate frequent itemset size:6


In [11]:
print(candidate_itemsets)
print(former_candidate)

['100', '101', '102', '103', '97', '98', '99', '100_101', '100_98', '101_102', '101_97', '101_98', '101_99', '102_103', '102_97', '102_98', '102_99', '103_99', '97_98', '97_99', '98_99']
['100_101', '100_98', '101_102', '101_97', '101_98', '101_99', '102_103', '102_97', '102_98', '102_99', '103_99', '97_98', '97_99', '98_99']


In [13]:
possible_item_set = []
[possible_item_set.extend(i.split('_')) for i in former_candidate]
possible_item_set = list(set(possible_item_set))

In [22]:
itemset_size = 3
itemsets_checked = []
for smaller_set_string in former_candidate:
    smaller_set = smaller_set_string.split('_')
#     print("smaller set:", smaller_set)
    possible_to_add = set(possible_item_set) - set(smaller_set)
#     print("Possible to add:", possible_to_add)
    for item_to_add in possible_to_add:
        new_itemset = sorted(smaller_set + [item_to_add])
        new_itemset_string = '_'.join(new_itemset)
        if new_itemset_string not in itemsets_checked:
            itemsets_checked.append(new_itemset_string) # reduce replicated checking
            print("\nNew_itemset:",new_itemset)
            immediate_subset = ['_'.join(sorted(i)) for i in combinations(new_itemset,itemset_size-1)]
    #         print("Immediate subset:",immediate_subset)
            all_subset_in_former = all([i in former_candidate for i in immediate_subset])
            print("All immediate subsets are frequent:",all_subset_in_former)
            if all_subset_in_former:
                for basket in clean_baskets:
                    if all([i in basket for i in new_itemset]):
                        itemset_cnt.update([new_itemset_string])


New_itemset: ['100', '101', '102']
All immediate subsets are frequent: False

New_itemset: ['100', '101', '97']
All immediate subsets are frequent: False

New_itemset: ['100', '101', '103']
All immediate subsets are frequent: False

New_itemset: ['100', '101', '98']
All immediate subsets are frequent: True

New_itemset: ['100', '101', '99']
All immediate subsets are frequent: False

New_itemset: ['100', '102', '98']
All immediate subsets are frequent: False

New_itemset: ['100', '97', '98']
All immediate subsets are frequent: False

New_itemset: ['100', '103', '98']
All immediate subsets are frequent: False

New_itemset: ['100', '98', '99']
All immediate subsets are frequent: False

New_itemset: ['101', '102', '97']
All immediate subsets are frequent: True

New_itemset: ['101', '102', '103']
All immediate subsets are frequent: False

New_itemset: ['101', '102', '98']
All immediate subsets are frequent: True

New_itemset: ['101', '102', '99']
All immediate subsets are frequent: True

N

In [ ]:
# frequent itemsets with size >= 3
for itemset_size in range(3, max_itemset_size + 1):
    itemset_cnt = Counter()
    possible_item_set = []
    [possible_item_set.extend(i.split('_')) for i in former_candidate]
    possible_item_set = list(set(possible_item_set))
    for smaller_set_string in former_candidate:
        smaller_set = smaller_set_string.split('_')
        possible_to_add = set(possible_item_set) - set(smaller_set)
        for item_to_add in possible_to_add:
            new_itemset = sorted(smaller_set + [item_to_add])
            immediate_subset = ['_'.join(sorted(i)) for i in combinations(new_itemset,itemset_size-1)]
            all_subset_in_former = all([i in former_candidate for i in immediate_subset])
            if all_subset_in_former:
                for basket in clean_baskets:
                    if all([i in basket for i in new_itemset]):
                        itemset_cnt.update([new_itemset_string])
    former_candidate = [i for i in itemset_cnt if itemset_cnt[i] >= support]
    candidate_itemsets.extend(former_candidate)
    if len(former_candidate)<2:
        print("True possible maximum frequent itemset size: %d" % itemset_size)
        break